In [98]:
import pyndri
import collections
import io

index = pyndri.Index('index/')

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                print("Max capacity reached... Breaking...")
                break

    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    topics = (parse_topics([f_topics]))

In [102]:
topics['51']

'Airbus Subsidies'

In [195]:
import os
import sys
from collections import defaultdict 

# Run evaluation bash script...
# subprocess.call(["./assignment_evals.sh"])


# ... And retrieve again
eval_data = {}
query_ids = {}
# For each file in given directory...
for file in os.listdir("run/TREC_results_on_test_set"):
    
    # Skip directories, DS_Store ghost files and .run files:
    if file != ".DS_Store" and file[-4:] != ".run":
        print(file)
        method = file.split("_")[0]
        with open("run/TREC_results_on_test_set/" + file) as file_obj:
            content = [line.strip().split() for line in file_obj.readlines()]
            
            content_dict = defaultdict(list)
            query_ids, query_len = [], []
            
            for metric, query, score in content:
                content_dict[metric].append(score)
                if query not in query_ids:
                    query_ids.append(query)
            
            for query_id in query_ids:
                if query_id != 'all':
                    query_len.append(len(topics[query_id].split()))
            
            query_len.append(-1)
            content_dict["method"] = len(query_ids) * [method]
            content_dict["query_ids"] = query_ids
            content_dict["query_len"] = query_len
            eval_data[method] = content_dict

absolute_0.5_test_queries_len_C_is_num_words.txt
BM25_testqueries.txt
dirichlet_500_test_queries_len_C_is_num_words.txt
jelinek_0.1_test_queries_len_C_is_num_words.txt
LSI_test_topics_500.txt
LSITFIDF_test_topics_500.txt
PLM_Gaussian_500_test_queries_len_C_is_num_words.txt
TF-IDF_.txt
W2V_test_5eps.txt


In [219]:
import pandas as pd


top_five_frames = []

models = list(eval_data.keys())
for model in models:
    data = pd.DataFrame(eval_data[model])
    data = data[data.query_ids != 'all']
    data = data[data.recall_1000 != '0.0000']
    data = data[['method', 'query_ids', 'query_len', 'num_rel', 'recall_1000', 'recall_500', 'ndcg_cut_10']]
    data = data.sort_values(by=['ndcg_cut_10'], ascending=False).head(5)
    top_five_frames.append(data)
    
    
#hard_queries = 
data = pd.concat(top_five_frames)

data


,method,query_ids,query_len,num_rel,recall_1000,recall_500,ndcg_cut_10
83,absolute,51,2,32,1.0000,1.0000,1.0000
75,absolute,193,3,120,0.5500,0.5167,1.0000
84,absolute,52,3,278,0.8597,0.7626,1.0000
87,absolute,56,5,284,0.9366,0.8169,1.0000
52,absolute,163,5,78,0.9872,0.9872,1.0000
52,BM25,163,5,78,0.9615,0.9615,1.0000
87,BM25,56,5,284,0.7993,0.6831,1.0000
66,BM25,183,3,61,1.0000,0.9836,0.9364
75,BM25,193,3,120,0.5250,0.5167,0.9306
82,BM25,200,9,39,0.9487,0.8718,0.9306


In [200]:
with open('mytable.tex','w') as tf:
    tf.write(data.to_latex())

In [211]:
data = pd.DataFrame(eval_data['BM25'])
data = data[data.query_ids != 'all']
data = data[['method', 'query_ids', 'query_len', 'num_rel', 'recall_1000', 'recall_500', 'ndcg_cut_10']]
data = data.sort_values(by=['recall_1000'])
data

,method,query_ids,query_len,num_rel,recall_1000,recall_500,ndcg_cut_10
96,BM25,66,3,2,0.0000,0.0000,0.0000
95,BM25,65,3,0,0.0000,0.0000,0.0000
4,BM25,105,2,33,0.0000,0.0000,0.0000
8,BM25,109,3,8,0.0000,0.0000,0.0000
115,BM25,91,7,5,0.0000,0.0000,0.0000
102,BM25,73,5,106,0.0189,0.0094,0.0000
74,BM25,191,5,127,0.0472,0.0079,0.0000
113,BM25,87,8,52,0.0577,0.0577,0.0000
107,BM25,80,4,168,0.0595,0.0357,0.0000
97,BM25,67,4,442,0.0611,0.0362,0.0784
